In [208]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [209]:
import torch

In [210]:
import sys
sys.path.insert(0, '/media/mistertandon/DATA/git_repos/ai/01-llm-rsbt/ch_02')

from ch02_26_data_sampling import GPTDatasetV1, create_dataloader_v1

In [211]:
import sys
sys.path.insert(0, '/media/mistertandon/DATA/git_repos/ai/01-llm-rsbt/ch_03')

from ch_03_3_1_simple_self_attention import naive_softmax

In [212]:
vocab_size = 50257
out_dim = 3
token_embedding_layer = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=out_dim)
print(token_embedding_layer)

Embedding(50257, 3)


In [213]:
raw_text = "millionaire journey start with first step"

In [214]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=6, stride=6, shuffle=False, drop_last=False, num_workers=0)
data_iter = iter(dataloader)
print(data_iter)

In [215]:
inputs, targets = next(data_iter)

In [216]:
print(inputs)

tensor([[14100,  7626,  7002,   923,   351,   717]])


In [217]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([1, 6, 3])


In [218]:
token_embeddings[0, 0]

tensor([ 1.3130, -1.0238,  0.0222], grad_fn=<SelectBackward0>)

In [219]:
position_embedding_layer = torch.nn.Embedding(6, 3)
print(position_embedding_layer)


Embedding(6, 3)


In [220]:
positional_embeddings = position_embedding_layer(torch.arange(6))
print(positional_embeddings.shape)

torch.Size([6, 3])


In [221]:
_inputs_embeddings = token_embeddings + positional_embeddings
inputs_embeddings = _inputs_embeddings.squeeze(0)
print(inputs_embeddings.shape)
print(inputs_embeddings)

torch.Size([6, 3])
tensor([[ 1.6234, -1.5950, -0.4331],
        [ 1.7970, -1.0687, -0.2073],
        [ 0.1557, -0.6514,  1.2317],
        [ 1.2563, -1.8800, -0.2736],
        [-0.4797,  0.9280, -3.7251],
        [ 1.4200, -1.3682,  0.3402]], grad_fn=<SqueezeBackward1>)


No we'll try to compute attention score corresponding to q^2 i.e. journey

attention_score_2 is un-normalized attention score represneted by omega for 2nd element (At index 1)

In [222]:
query = inputs_embeddings[1]
attention_score_2 = torch.empty(inputs_embeddings.shape[0])
for idx, inputs_embedding in enumerate(inputs_embeddings):
    attention_score_2[idx] = torch.dot(query, inputs_embedding)

print(attention_score_2)

tensor([ 4.7115,  4.4142,  0.7207,  4.3234, -1.0817,  3.9435],
       grad_fn=<CopySlices>)


attention_weight_2_normalized is normalized attention score (Known as attention weigth Represented by Alpha) for 2nd element (At index 1)

In [ ]:
attention_weight_2_normalized = torch.softmax(attention_score_2, dim=0)
print(attention_weight_2_normalized)

tensor([0.3440, 0.2556, 0.0064, 0.2334, 0.0010, 0.1596],
       grad_fn=<SoftmaxBackward0>)


In [224]:
attention_weight_2_naive_normalized =naive_softmax(attention_score_2)
print(attention_weight_2_naive_normalized)
print(attention_weight_2_naive_normalized.sum())

tensor([0.3440, 0.2556, 0.0064, 0.2334, 0.0010, 0.1596],
       grad_fn=<DivBackward0>)
tensor(1., grad_fn=<SumBackward0>)
